<a href="https://colab.research.google.com/github/pourzand/CSCI499/blob/main/HW1_Unsolved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Overall guidelines:
- Only change the cells which say:
```# INSERT CODE/TEXT HERE```
- Except Q1, other content will be covered in class starting Jan 17.


# Q1. Probability (1)

Say we have three random variables $A$ and $B$ and $C$. Note that we’re using standard probability
theory notation where $P(A, B) = P(B, A)$, which simply means the joint probability of both A
and B occurring.

### Q1.1
(0.5 point)

Which of the following statements is always true?
1. $P(A|B) = P(B|A)$
2. $P(A, B) = P(A) + P(B)$
3. $P(A, B, C) = P(A)P(B)P(C)$
4. $P(A|B) = max(P(A),P(B))$
5. $P(A, B, C) = P(A)P(C)$
6. $P(A, B) = P(B|A)P(A)$
7. $P(A, B, C) = P(A)P(B|A)P(C|A, B)$
8. $P(A)=\sum_{b ∈ \text{domain}(B)} P(A, B=b)$
9. $P(A)=\sum_{b ∈ \text{domain}(B)} P(A|B=b)P(B=b)$
10.$\log(1/P(A)) = 1 - log P(A)$

### Q1.2
(0.5 point)

Now assume that A, B, and C are all independent of each other. Which of the above statements
are now true?

**\# INSERT TEXT HERE**

Answer the two subparts here by listing which statement (just the statement numbers) are true.

# Q2. N-gram language model (3)

Create a 5-gram language model trained on the Shakespeare dataset. Store the relevant probabilities P(word|context) in python dictionaries. Do not use any smoothing or back-off (until Question 4). Pay special attention to beginning and end of sequences in the modeling process.

In [ ]:
import requests
import collections
import random
import sys
import numpy as np
from tqdm.notebook import tqdm
import math

In [ ]:
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
response.raise_for_status() # Raise an exception for invalid HTTP status codes
text_data = response.text
len(text_data), text_data[:100]


(1115394,
 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou')

In [ ]:
# sample
pos = random.randint(0, len(text_data) - 1000)
print(text_data[pos:pos+100])

 the beetle brows shall blush for me.

BENVOLIO:
Come, knock and enter; and no sooner in,
But every 


In [ ]:
# preprocessing - do not change
text_data = response.text
text_data = text_data.replace(',',' , ').replace('.',' . ').replace('?',' ? ').replace('!',' ! ')
text_data = text_data.replace('  ', ' ')
text_data = text_data.replace('\n\n','\n').replace('\n',' </s> <s> ')
text_data = '<s> ' + text_data + ' </s>'
len(text_data), text_data[:100]

(1451204,
 '<s> First Citizen: </s> <s> Before we proceed any further , hear me speak .  </s> <s> All: </s> <s> ')

In [ ]:
test_data = text_data[-10_000:]
text_data = text_data[:-10_000]
len(text_data), len(test_data)

(1431204, 10000)

In [ ]:
# split words
data = text_data.split(' ')
len(data), data[:10]

(311889,
 ['<s>',
  'First',
  'Citizen:',
  '</s>',
  '<s>',
  'Before',
  'we',
  'proceed',
  'any',
  'further'])

In [ ]:
# UPDATE: Removing unseen tokens from test_data
vocab = set(data)
test_data = ' '.join([_ for _ in test_data.split(' ') if _ in vocab])
print(test_data[:100])

or dost unwillingly </s> <s> What I command , I'll rack thee with old cramps ,  </s> <s> Fill all th


In [ ]:
five = collections.defaultdict(lambda:0)
four = collections.defaultdict(lambda:0)
tri = collections.defaultdict(lambda:0)
bi = collections.defaultdict(lambda:0)
uni = collections.defaultdict(lambda:0)

# INSERT CODE HERE


In [ ]:
# Evaluation - Do not change
assert five[('Before','we','proceed','any','further')] == 1.0 # prob of last given prev 4
assert four[('<s>','First','Citizen:','</s>')] == 1.0 # prob of last given prev 3
assert tri[('<s>','First','Citizen:')] == 0.172 # prob of last given prev 2
assert round(bi[('First','Citizen:')],3) == 0.169 # prob of last given prev 1
assert round(uni[('Citizen:')],5) == 0.00031 # prob of last

Using the dictionaries containing unigram, bigram, ... five-gram probabilities, return probability P(word|context) as a single scalar. Here context is a list of previous tokens, e.g., [`You`, `may`, `do`, `as`, `you`] and word is a single potential next word, e.g., `like`.

In [ ]:
def get_prob_from_lm(context=[], word=""):
  return 0
  # INSERT CODE HERE


In [ ]:
get_prob_from_lm(["<s>", "First"], "Citizen:")

# Q3. Evaluate Perplexity (2.5)

Now let's evaluate the models quantitively using the intrinsic metric **perplexity**.

Recall perplexity is the inverse probability of the test text
$$\text{ppl}(w_1, \dots, w_n) = p(w_1, \dots, w_n)^{-\frac{1}{N}}$$

For an n-gram model, perplexity is computed by
$$\text{ppl}(w_1, \dots, w_n) = (\prod_i p(w_{i+n}|w_i^{i+n-1})^{-\frac{1}{N}}$$

To get rid of numerical issue, we usually compute through:
$$\text{ppl}(w_1, \dots, w_n) = \exp(-\frac{1}{N}\sum_i \log p(w_{i+n}|w_i^{i+n-1}))$$


*Input:*

+ **prob_function**: the `get_prob_from_lm` you implemented in Q2.
+ **data**: test data is a string of text containing multiple words.

*Output:*
+ the perplexity of given data under a language model defined by **prob_function**

Note that you do NOT need to optimize the language model in any way so as to minimize perplexity. Your reported perplexity will have no correlation with your score on this assignment, as far as it is implemented correctly.

In [ ]:
def compute_perplexity(prob_function=get_prob_from_lm, data=test_data):
  ...
  # Hint: something = prob_function(context, word)
  # INSERT CODE HERE

In [ ]:
simple_data = 'Before we proceed any further , hear me speak .'
compute_perplexity(data=simple_data)

Can the perplexity of a sentence under any 5-gram model be 1? Explain.

**\# INSERT TEXT HERE**


In [ ]:
compute_perplexity(data=test_data)

**Explain** the resulting perplexity above on test data.
Conjecture (in text, not code) how it could be improved?

**\# INSERT TEXT HERE**

# Q4. Interpolation Smoothing (1)

Using the dictionaries containing unigram, bigram, ... five-gram probabilities, return probability P(word|context) where context is the previous string, e.g., `You may do as you` and word is a single potential next word, e.g., `like`.

Unlike in Q2, this time use interpolation smoothing as discussed in class.

In [ ]:
l1=0.2; l2=0.2; l3=0.2; l4=0.2; l5=0.2
# lambdas needed for Interpolation - you do NOT need to change these for the assignment
def get_prob_from_lm_with_interpolation(context=[], word=""):
  # INSERT CODE HERE
get_prob_from_lm_with_interpolation(['Before', 'we', 'proceed', 'any'],
                                    'further')

Now we re-evaluate perplexity with the new probability estimates (interpolated).

In [ ]:
compute_perplexity(prob_function=get_prob_from_lm_with_interpolation,
                   data=simple_data)

In [ ]:
compute_perplexity(prob_function=get_prob_from_lm_with_interpolation,
                   data=test_data)

# Q5. Text Generation (2.5)

Our final goal is to generate/infer/sample from the language model that we have created.
Given a prefix or prompt, the model can generate text according to estimated next word probabilities (similar to any LLM, e.g., GPT).

Complete the following function using the dictionaries already available to sample (you may use `random.choice` function) from the distribution of possible next tokens. Remember to randomly sample (from a weighted distribution over all possible next words in the vocabulary) instead of picking only the most likely next word.

Input:
- dictionaries `uni`, `bi`, `tri`, `four`, `five`
- **n_generate**: number of maximum words to infer. E.g., `2`
- **prefix**: the context on which to condition text generation. E.g., `as you`

Output: string output containing prefix and generated words separated by whitespace. E.g., `as you like it`

In [ ]:
prefix = 'as you say you' # "hear me"
def generate(prob_fn=get_prob_from_lm_with_interpolation,
             prefix=prefix, n_generate=20):
  # INSERT CODE HERE

In [ ]:
generate(prob_fn=get_prob_from_lm)

In [ ]:
generate(prob_fn=get_prob_from_lm_with_interpolation)

Compare the outputs of the two language models. Why do you think a difference exists?

**\# INSERT TEXT HERE**

# The End
Voila! If you were able to complete the assignment, you have successfully created a language model in barebones Python! You have trained it on a text corpus and can use it to generate arbitrary text just like GPT-3!